# Simulation of TLS handshake

In [61]:
""" # Please remove the triple quotes in this line if the pip installation of the libraries is needed
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
%pip install pycryptodome
%pip install cryptography
%pip install requests --disable-pip-version-check """;

**Client side**

The client generates a nonce of size 32 bytes. The client sends the nonce to the server along with a flag that is set to 1 if RSA will be used for key exchange, and set to zero if Diffie-Hellman will be used

In [62]:
import random
from Crypto.Random import get_random_bytes

# generate random nonce, size 32 bytes → 256-bit
nonce_client = get_random_bytes(32)    #RB
# specify exchange method → 0=Diffie-Hellman / 1=RSA
exchange_method_client=random.randint(0,1)
print("Nonce-client: ",nonce_client)
print("Exchange method-client:",exchange_method_client)

Nonce-client:  b'\xa8oS\xb4\xe4\xd3\xe1R\x1aS\x0b\xb7\xf7\xc6)\x8eQ\xe8\xa6\xcb\xbbt\xe2\x13"\xc2\xea\xbd\x8f\xe1R\xfc'
Exchange method-client: 1


**Server side**

The server responds with another nonce of size 32 bytes and a flag indicating the choice of key exchange algorithm.

In [63]:
# generate random nonce, size 32 bytes → 256-bit
nonce_server = get_random_bytes(32)     #RS
# specify exchange method → 0=Diffie-Hellman / 1=RSA
exchange_method_server=random.randint(0,1)
print("Nonce-server: ",nonce_server)
print("Exchange method-server:",exchange_method_server)

Nonce-server:  b"\xb1\x0c1\xdb\xbe\x9e`\x8cn\xa0 \x02\xee\xdc'g\x19\x8d\xa5\x87\xdf\xe7\xefs_\x07o\xb29\x85\x14I"
Exchange method-server: 0


**Generate RSA KEY pair**

The server randomly generates a pair of RSA public/private keys and sends to the client {public key, hash(public key)}. This constitutes the certificate.

In [64]:
from Crypto.PublicKey import RSA
# generate RSA key pair
key = RSA.generate(2048)
publickey = key.publickey() # for later use
pu_key=key.publickey().exportKey()
priv_key=key.exportKey()
print(pu_key.decode())
# print(priv_key.decode())  #private key
# print(key.public_key().export_key().decode())
# print(f"Public key:  (n={hex(key.n)}, e={hex(key.e)})")
# print(f"Private key: (n={hex(key.n)}, d={hex(key.d)})")

-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA/JCtP5Rtz9ziKhPUipNE
pXbUuvLFENFn5LptuZSZ1xR5kTYOBbdF/5d51n0z1tjbQ5D6pE5xLdXdxAxxVODy
z0wCaJzZ8A0GdHF/j7y7hYNA3M9at/z8/ATPbOkGuW+vJk5JGKaxsH9YdhqCdCCd
c30keKfvb9tulE//596jUfBN2t6FVgXIypZKvN6YbS+PmWQ0gZ9MUu4NrGR0x/Ei
9BLhbiGVfkdwBw6kKHSHq1FcRCAbrI/oL0+a4hHRpzfQRn3/2scn16DVbZrhmLHF
77id3bR88ngcHyNneCJw+XuN5HOKxgEWvIh3wHaVUCu6Tkt3kjmSUxN7zcTs+j6i
uwIDAQAB
-----END PUBLIC KEY-----


**Key hashing-server**

In [65]:
import hashlib
from hashlib import sha256

# Hash key 
pk_client=key.public_key().export_key()
my_sha = hashlib.sha256(pk_client)
hash_client = int.from_bytes(my_sha.digest(),byteorder='big')
print("Sent to client: {public key, hash(public key)} ",'\n'+'↓↓↓↓↓↓↓↓↓↓↓↓','\n')
print("Public key :",'\n',pk_client.decode(),'\n')
print("Generated hash :",'\n',hash_client)
# signature = pow(hash, key.d, key.n)
# print("Signature:", hex(signature))

Sent to client: {public key, hash(public key)}  
↓↓↓↓↓↓↓↓↓↓↓↓ 

Public key : 
 -----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA/JCtP5Rtz9ziKhPUipNE
pXbUuvLFENFn5LptuZSZ1xR5kTYOBbdF/5d51n0z1tjbQ5D6pE5xLdXdxAxxVODy
z0wCaJzZ8A0GdHF/j7y7hYNA3M9at/z8/ATPbOkGuW+vJk5JGKaxsH9YdhqCdCCd
c30keKfvb9tulE//596jUfBN2t6FVgXIypZKvN6YbS+PmWQ0gZ9MUu4NrGR0x/Ei
9BLhbiGVfkdwBw6kKHSHq1FcRCAbrI/oL0+a4hHRpzfQRn3/2scn16DVbZrhmLHF
77id3bR88ngcHyNneCJw+XuN5HOKxgEWvIh3wHaVUCu6Tkt3kjmSUxN7zcTs+j6i
uwIDAQAB
-----END PUBLIC KEY----- 

Generated hash : 
 25721024698927385901025868719334761964613471435580153800216506663182367606818


In [66]:
# hash2 = int.from_bytes(my_sha.digest(),byteorder='big')
# hashFromSignature = pow(signature, key.e, key.n)
# print("Signature valid:", hash2 == hashFromSignature)

**Key re-hashing-client**

Client re-hashes the public key and and compares with the hash that is sent. If they match the client proceeds to Step 5. Otherwise, the process ends here with an error message.

In [67]:
# re-Hash key 
my_sha = hashlib.sha256(pk_client)
hash_server = int.from_bytes(my_sha.digest(),byteorder='big')
if (hash_client==hash_server):
    print("hash match")
else:
    print("hash missmatch")

hash match


# RSA-TLS

In case RSA was chosen, then the client will generate a pre-master secret and encrypt it with the public key of the server that was already sent over by the server.

In RSA key exchange the public key is used in the client to encrypt the pre-master secret and the private key is used in the server to decrypt it.

In [68]:
from Crypto.Cipher import PKCS1_OAEP
# encrypt the Pre-master secret with the public key of the server
encryptor = PKCS1_OAEP.new(publickey)
pre_masterSecret_client = get_random_bytes(32)     #PS generated from the client 
pre_masterSecret_server = get_random_bytes(32)     #PS generated from the server 
encrypted = encryptor.encrypt(pre_masterSecret_client)
print(encrypted)

b'\xc7\xb9\xa6\x9b\xc6_ \xd8\xcb\x81\x9e\x88\x8fL\x14\xeb\xe7>\xfe-\xbf\x90\x01]\xb2\xec\xcf7\x83\xa9\x97\xf19|x\x96BW\xbb\xcaG-\x10\x9e\xbb\x82\x80\x84f\xfe\x19\xcb/\x9f\xb4\xaa\x87\xa1\x87p\x89\x9d\n\xc7Kx6\xa9\xa91,\x9d~\xdf&\xd7\xd0\xe2\xc6\xee4"x\x9d\x06\x1f#]\xf7\xf0\x06\xb4\xf2t\xe0[\xcf\xa6\xfb\\\xf1^\x0fwi-\xd6\x89,\xf25\xa4\x1a\xd1\xbc\x99\x8a\x9d\xcb\x03\x17\xbf\x16@\x0e\xc7\xdd\xbbG{\xae\xd91\x12\xeb$\x98+2\xbe\xa2~\xecE\x12\xb5n\x1e\xe5\xcaJ\xb3\xc8{\xae\xaf\xc17\x83y[\x0c\x9b{\xd8\r\x98\xaa:\xa8K<\x81w\xcf\xbb\xf81\xd0\xcb\xc5=\x9eN\x1f\x98G\xd84\xef\xc6\xcb\x0b\'\xa6\xc5\xfd\xff\x88\xe3\xcaO\xca\xee\xb7\xe5e\x80\xf8Z\xb5\xbf\x92\xd4\xa1\xed\x19 V\'\xb0N\xd7JH\x10\x9a\xa6\xcd\x9d"\x17\xf7\xdd\xac\x15<I;\xfd\xd9\xec\xa0\xe6\x13\xc5Eo\x8eC\xdd?\xf9(\xb1c'


The server will decrypt it using his private key and validate the results

In [69]:
import hmac
# master secret generated at the client side (CC -Cipher client)
def prf(secret,seed,numblocks):
    seed=seed
    output = b''
    a=hmac.new(secret,msg=seed,digestmod=hashlib.sha256).digest()
    for j in range(numblocks):
        output += hmac.new(secret,msg=a+seed,digestmod=hashlib.sha256).digest()
        a=hmac.new(secret,msg=a,digestmod=hashlib.sha256).digest()
    return output

def master_secret(pms,client_random,server_random):
    out=prf(pms,client_random+server_random,2)
    return out[:48]

def keyblock(ms,client_random,server_random):
    u=prf(ms,server_random+client_random,4)
    return u

label=b'master secret_client'
cipher_client=prf(pre_masterSecret_client,label+nonce_client+nonce_server,2)
print("Cipher Client - CC: ",'\n'*2,cipher_client)

Cipher Client - CC:  

 b'\x82\xd9\xbbld\x19\xa4P\xb2\xaf\xe7x\xa5\xfeWj\xc5\xde\x8b\xee\xb2\x95\x03\n.\xcd\x01\x11Nw\xd9\x0f\xd1ht+Y\x9f=\xb6\x0b\\\xe7\xcf\xaf\x86\xef\xc2\x05\xf7p\xd7\xe4\xb2\xd9Q\xf9\xb9lZ\xedp{\x13'


In [70]:
label=b'master secret_server'
cipher_server=prf(pre_masterSecret_server,label+nonce_client+nonce_server,2)
print("Cipher Server - CS: ",'\n'*2,cipher_server)

Cipher Server - CS:  

 b'\xbf\xa6\x97\xe3E\xf8f\xcb\xfb\x94Kz\xf7\xa4\x9c\\\xfc\xb5\x193\x91C\xde\xbe\xc9w\xc7\xb2\xe81h\xc0HE)\x89i\x06f\xf2L\xe2J\x8d)\x14w\xd9T\xd54\xc7\xf1\xc3\x1e\x14H\xf7\xbd%3D\xd5\xa6'


# Diffie-Hellman TLS

In [83]:
import random

p = random.randint(1, 100)
g = random.randint(1, 100)
a = random.randint(1, 100)
b = random.randint(1, 100)
A = ((pow(g, a)) % p)           
B = ((pow(g, b)) % p)
pre_master_client = ((pow(B, a)) % p)
pre_master_server = ((pow(A, b)) % p)
print("PS (Client) = ", str(pre_master_client))
print("PS (Server)",str(pre_master_server))

PS (Client) =  10
PS (Server) 10
